**Authors:** Anowar Shajib, David Law

*Adapted from the [notebook](https://github.com/JWST-Templates/Notebooks/blob/main/MIRI_MRS_reduction_SPT0418-47_PAH_ch3long.ipynb) by the TEMPLATES team (J. Spilker, K. A. Phadke, D. Law).*

In this notebook we process our data through the Spec2 pipeline in order to produce 
Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
    
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html

During stage 2 of the pipeline, the countrate (slope) image products from stage 1, which have units of DN/s, are converted to units of surface brightness (MJy/sr) for both extended and point sources (as of DMS build 9.3/CAL_VER 1.10.2). 

If Pixel-based background subtraction was chosen, this will be applied during Spec2. 
Doing the Pixel based background subtraction requires an association file to be created to associate the background files with individual science files.

In [1]:
import numpy as np
from tqdm import tqdm
import pathlib
from scipy.ndimage import median_filter
from util import *

# JWST pipe version = 1.12.5
# STCAL version = 1.4.4


In [2]:
run_pipeline = True
run_on_nsclean_data = True
do_extra_processing = True
subtract_pixel_based_background = False
# Pixel-based background subtraction in spec2 (direct pixel subtraction) -Deep background exposures needed to not add noise.

if run_on_nsclean_data:
    stage1_directory = stage1_nsclean_directory
    stage2_directory = stage2_nsclean_directory
else:
    stage1_directory = stage1_processed_directory
    stage2_directory = stage2_directory

In [3]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def run_spec2_pipeline(file_name, output_directory, no_cubes=False):
    """
    Run the Spec2Pipeline on a given file.

    :param file_name: str, name of fits file to run pipeline on
    :param output_directory: str, path to output directory
    :param no_cubes: bool, if True, skip cube building and 1d spectral extraction
    :return: None
    :outputs:
    """
    print("Running Detector2Pipeline on {0:s}...".format(file_name))

    # Set default configuration from CRDS param reference files.
    # -This is required when running the pipeline in a function.
    crds_config = Spec2Pipeline.get_config_from_reference(file_name)
    spec2 = Spec2Pipeline.from_config_section(crds_config)

    spec2.output_dir = output_directory

    # Assign_wcs overrides
    # spec2.assign_wcs.override_distortion = 'myfile.asdf'
    # spec2.assign_wcs.override_regions = 'myfile.asdf'
    # spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    # spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Background overrides were set up above
    if subtract_pixel_based_background:
        spec2.bkg_subtract.skip = False
    else:
        spec2.bkg_subtract.skip = True

    # Flatfield overrides
    # spec2.flat_field.override_flat = 'myfile.fits'

    # Straylight overrides
    # spec2.straylight.override_mrsxartcorr = 'myfile.fits'

    # Fringe overrides
    # spec2.fringe.override_fringe = 'myfile.fits'

    # Photom overrides
    # spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    # spec2.cube_build.override_cubepar = 'myfile.fits'

    # Extract1D overrides
    # spec2.extract_1d.override_extract1d = 'myfile.asdf'
    # spec2.extract_1d.override_apcorr = 'myfile.asdf'

    # Overrides for whether or not certain steps should be skipped
    # spec2.assign_wcs.skip = False
    # spec2.bkg_subtract.skip = True
    # spec2.flat_field.skip = False
    # spec2.srctype.skip = False
    # spec2.straylight.skip = False
    # spec2.fringe.skip = False
    # spec2.photom.skip = False
    # spec2.residual_fringe.skip = False #does a residual fringe correction across the entire cube
    # spec2.cube_build.skip = False
    # spec2.extract_1d.skip = False

    spec2.flat_field.override_sflat = "../data/sflat_drl.fits"

    # Run pixel replacement code to extrapolate values for otherwise bad pixels.
    # This can help mitigate small 5-10% negative dips in spectra of bright sources.
    spec2.pixel_replace.skip = True
    # ajshajib: David set this to False, but that throws an error that a numpy array
    # down the line is empty. So, skipping this step for now.
    # spec2.pixel_replace.algorithm='mingrad'

    # This nocubes option allows us to skip the cube building and 1d spectral extraction for individual
    # science data frames, but run it for the background data (as the 1d spectra are needed later
    # for the master background step in Spec3)
    if no_cubes:
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True

    # Some cube building options
    # spec2.cube_build.weighting='drizzle'
    # spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky

    spec2.save_results = True
    spec2(file_name)

In [4]:
rate_files = sorted(glob.glob(stage1_processed_directory + "/*nrs1_rate.fits"))

len(rate_files)

18

In [ ]:
# Step through each of the science files, using relevant associated backgrounds in spec2 processing
# The background files are used in this step to perform pixel-based background subtraction (if desired)
# Otherwise Background subtraction is done later with Spec3 files
if run_pipeline:
    for file in tqdm(rate_files):
        run_spec2_pipeline(file, output_directory=stage2_directory, no_cubes=True)
else:
    print("Skipping Spec2 processing for SCI data")

  0%|          | 0/18 [00:00<?, ?it/s]

Running Detector2Pipeline on /Users/ajshajib/Research/1131_JWST/data/stage1_processed/jw01794001001_02101_00001_nrs1_rate.fits...


2023-12-17 02:13:04,084 - CRDS - ERROR -  Error determining best reference for 'pars-pixelreplacestep'  =   Unknown reference type 'pars-pixelreplacestep'
2023-12-17 02:13:04,095 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2023-12-17 02:13:04,095 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-12-17 02:13:04,096 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-12-17 02:13:04,097 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2023-12-17 02:13:04,097 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2023-12-17 02:13:04,098 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2023-12-17 02:13:04,100 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2023-12-17 02:13:04,101 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2023-12-17

Running Detector2Pipeline on /Users/ajshajib/Research/1131_JWST/data/stage1_processed/jw01794001001_02101_00002_nrs1_rate.fits...


2023-12-17 02:16:22,055 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-12-17 02:16:22,056 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-12-17 02:16:22,057 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2023-12-17 02:16:22,058 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2023-12-17 02:16:22,058 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2023-12-17 02:16:22,060 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2023-12-17 02:16:22,061 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2023-12-17 02:16:22,061 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2023-12-17 02:16:22,062 - stpipe.Spec2Pipeline.master_background_mos.barshadow - INFO - BarShadowStep instance created.
2023-12-17 02:1

...

2023-12-17 02:24:47,224 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-12-17 02:24:47,225 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-12-17 02:24:47,226 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2023-12-17 02:24:47,226 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2023-12-17 02:24:47,227 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2023-12-17 02:24:47,229 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2023-12-17 02:24:47,229 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2023-12-17 02:24:47,230 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2023-12-17 02:24:47,230 - stpipe.Spec2Pipeline.master_background_mos.barshadow - INFO - BarShadowStep instance created.
2023-12-17 02:2

Running Detector2Pipeline on /Users/ajshajib/Research/1131_JWST/data/stage1_processed/jw01794001001_04101_00002_nrs1_rate.fits...


2023-12-17 02:27:17,688 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2023-12-17 02:27:17,688 - stpipe.Spec2Pipeline.wfss_contam - INFO - WfssContamStep instance created.
2023-12-17 02:27:17,689 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2023-12-17 02:27:17,689 - stpipe.Spec2Pipeline.pixel_replace - INFO - PixelReplaceStep instance created.
2023-12-17 02:27:17,690 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2023-12-17 02:27:17,692 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2023-12-17 02:27:17,692 - stpipe.Spec2Pipeline.extract_1d - INFO - Extract1dStep instance created.
2023-12-17 02:27:17,905 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline running with args ('/Users/ajshajib/Research/1131_JWST/data/stage1_processed/jw01794001001_04101_00002_nrs1_rate.fits',).
2023-12-17 02:27:17,913 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline parameters are: {'pre_hooks':

Running Detector2Pipeline on /Users/ajshajib/Research/1131_JWST/data/stage1_processed/jw01794001001_04101_00003_nrs1_rate.fits...


2023-12-17 02:29:48,738 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2023-12-17 02:29:48,739 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2023-12-17 02:29:48,740 - stpipe.Spec2Pipeline.master_background_mos.barshadow - INFO - BarShadowStep instance created.
2023-12-17 02:29:48,740 - stpipe.Spec2Pipeline.master_background_mos.photom - INFO - PhotomStep instance created.
2023-12-17 02:29:48,741 - stpipe.Spec2Pipeline.wavecorr - INFO - WavecorrStep instance created.
2023-12-17 02:29:48,742 - stpipe.Spec2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2023-12-17 02:29:48,743 - stpipe.Spec2Pipeline.srctype - INFO - SourceTypeStep instance created.
2023-12-17 02:29:48,743 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2023-12-17 02:29:48,744 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2023-12-17 02:29:48,745 - stpipe.Spec2Pipel

# Extra processing to flag bad pixels again

In [ ]:
if do_extra_processing:
    files = np.array(sorted(glob.glob(stage2_directory + "/*nrs1_cal.fits")))

    for file in files:
        with fits.open(file) as hdu:
            sci = hdu["SCI"].data

            # Flag positive pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 2
            index = np.where(diff > 100)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)
            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            # Flag negative pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 0.5
            index = np.where(diff < -100)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)
            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            hdu["SCI"].data = sci
            name = pathlib.Path(file).name
            hdu.writeto(stage2_processed_directory + name, overwrite=True)